 ### Structure mapping

 Also, since the pointer member  _S.p_  is used in an array section of a   `map`  clause, the array storage of the array section on the device,   _S.p[:N]_ , is {attached} to the pointer member  _S.p_  on the device. Explicitly mapping the pointer member  _S.p_  is optional in this case.

 Note: The buffer arrays and the  _x_  variable have been grouped together, so that the components that will reside on the device are all together (without gaps). This allows the runtime to optimize the transfer and the storage footprint on the device.

In [ ]:

/*
* name:       target_struct_map.1c
* type:       C
* version: omp_5.0
*/
#include <stdio.h>
#include <stdlib.h>
#define N 100
#define BAZILLION 2000000

struct foo {
  char buffera[BAZILLION];
  char bufferb[BAZILLION];
  float x;
  float a, b;
  float *p;
};

#pragma omp declare target
void saxpyfun(struct foo *S)
{
  int i;
  for(i=0; i<N; i++)
    S->p[i] = S->p[i]*S->a + S->b;
}
#pragma omp end declare target

int main()
{
  struct foo S;
  int i;

  S.a = 2.0;
  S.b = 4.0;
  S.p = (float *)malloc(sizeof(float)*N);
  for(i=0; i<N; i++) S.p[i] = i;

  #pragma omp target map(alloc:S.p) map(S.p[:N]) map(to:S.a, S.b)
  saxpyfun(&S);

  printf(" %4.0f %4.0f\n", S.p[0], S.p[N-1]);
        //     4  202  <- output
  return 0;
}



 
In this example a pointer,  _p_ , is mapped in a  
 `target` ~ `data`  construct ( `map(p)` ) and remains  
persistent throughout the  `target` ~ `data`  region. The address stored 
on the host is not assigned to the device pointer variable, and  
the device value is not copied back to the host at the end of the 
region (for a pointer, it is as though  `map(alloc:p` ) is effectively 
used).  The array section,  _p[:N]_ , is mapped on both  `target`  
constructs, and the pointer  _p_  on the device is attached at the ning and detached at the end of the regions to the newly created 
array section on the device. 